### 导入所需库

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import abc
import xlrd
from xlrd import xldate_as_tuple
import os
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
from itertools import product
from statsmodels.tsa.arima_model import ARMA,ARIMA
import warnings
warnings.filterwarnings('ignore')

### 自定义函数

In [2]:
def read_from_excel(wb,sheet_name):
    sheet = wb.sheet_by_name(sheet_name)# 获取名为sheet
    dat = []  # 创建空list记录每行数据
    for row in range(sheet.nrows):  # 循环读取表格内容（每次读取一行数据）
        cells = sheet.row_values(row)  # 每行数据赋值给cells
        for col in range(sheet.ncols):
            value = sheet.cell(row, col).value
            if sheet.cell(row, col).ctype == 3: # ctype=3表示日期格式
                date = xldate_as_tuple(sheet.cell(row, col).value, 0)
                value = datetime(*date).strftime('%Y-%m-%d') # 将日期格式标准化
            cells[col] = value
        dat.append(cells) #把每次循环读取的数据插入到list
    df = pd.DataFrame(dat[1:],columns = dat[0])
    return df

In [3]:
def mom_to_yoy(df): 
    
    # 将环比数据转换为同比数据
    
    df['同比'] = 0
    df = df.reset_index(drop = True)
    for i in range(11,df.shape[0]):
        df.iloc[i,1] = (df.iloc[(i-11):(i+1),0]+100).prod()/100**11-100
 
    return df

In [4]:
def choose_best_ARMA(df):
    ps = range(0, 4)
    qs = range(0, 4)
    parameters = product(ps, qs)
    parameters_list = list(parameters)
    best_aic = float('inf')
    results = []
    for param in parameters_list:
        try:
            model = ARMA(df, order=(param[0], param[1])).fit()
        except ValueError:
            continue
        aic = model.aic
        if aic < best_aic:
            best_model = model
            best_aic = model.aic
            best_param = param
        results.append([param, model.aic])
    results_table = pd.DataFrame(results)
    results_table.columns = ['parameters', 'aic']
    return best_model

### 新建抽象类

In [5]:
class model(object,metaclass = abc.ABCMeta): # 抽象基类
    
    @abc.abstractmethod    
    def PPI_by_Industry(self,industry_names,verbose = 0):
        # 分别预测重要行业的PPI，再将其拟合成最终的PPI
        pass
    
    @abc.abstractmethod    
    def PPI_by_PMI(self,verbose = 0):
        # 通过PMI:主要原材料购进价格来预测PPI环比
        pass
    


### 新建继承类

In [6]:

class PPI_model(model):
    
    def __init__(self,workbook,date):
        self.workbook = workbook
        self.date = date
   
    def PPI_by_Industry(self,industry_names,verbose = 0):
        date2 = datetime.strptime(self.date,'%Y-%m-%d')                                    # 预测时间格式标准化
        nmonths = 1                                                                        # 预测的总月份数 
        PPI_by_industry_pre = np.zeros(shape=(nmonths,len(industry_names)))                # 新建数组来记录预测值
    
        ######## 对行业分别进行预测 ########
    
        for i,name in enumerate(industry_names):
            df = read_from_excel(self.workbook,name)                                        # 通过名称获取
            df_train = df[df.date < self.date]                                                   # 用2020年5月以前的数据建模
            df_test = df[df.date == self.date]                                                   # 预测2020年5月
            y = np.asarray(df_train.y,'float32')
            x = df_train.drop(['date','y'],axis = 1)
            x2 = sm.add_constant(x)                                                         # 添加常数1
            est = sm.OLS(y, x2).fit()                                                       # 建模  
            x_test = sm.add_constant(np.asarray(df_test.drop(['date','y'],axis = 1)),
                                     has_constant='add')                                    # 添加常数1
            PPI_by_industry_pre[:,i] = est.predict(x_test)                                  # 逐列添加每个行业的预测值       

            if verbose == 1:                                                                # 如果 verbose=1，输出模型细节
                print(name,
                  "的修正R方为",
                  '{:.2%}'.format(est.rsquared_adj))                                        # 输出修正R方
                summary = pd.DataFrame({
                                    '系数':est.params[1:],
                                    'p值':est.pvalues[1:]})                                  # 新建df记录系数和p值
                print(summary)                                                               # 输出系数和p值
        
    
        ######## 计算各行业的系数并拟合总体PPI ########
    
        df = read_from_excel(self.workbook,"综合")
        df.index = pd.DatetimeIndex(df.date)
        df = df.drop('date',axis = 1)
        df = df.apply(pd.to_numeric,axis = 1)
        x = np.zeros(shape = (df[df.index<date2].shape[0],len(industry_names)))
        for i,name in enumerate(industry_names):                                             # 按照顺序添加各行业PPI作为自变量
            x[:,i] = df[df.index<date2][name]
        x2 = np.asarray(sm.add_constant(x))                                                  # 添加常数1
        y = np.asarray(df[df.index<date2]['PPI环比'])
        est_coef = sm.OLS(y, x2).fit()                                                       # 建模  
        summary = pd.DataFrame({'系数':est_coef.params,
                                    'p值':est_coef.pvalues})                                  # 新建df记录各行业系数和p值
        print("即将输出各行业系数...")
        print(summary)                                                                        # 输出各行业系数和p值
        x_test_final = sm.add_constant(PPI_by_industry_pre,has_constant='add')
        PPI_mom_pre = est_coef.predict(x_test_final)
        PPI_mom = pd.DataFrame(np.hstack((y,PPI_mom_pre)),columns=['PPI环比'],
                               index = range(y.shape[0]+1))                                   # 最终PPI环比预测
    
        ######## 环比数据转为同比数据 ########  
        
        PPI_yoy = mom_to_yoy(PPI_mom)
        PPI_yoy.index = pd.date_range(start = df.index[0],periods = y.shape[0]+1,freq = 'M')
    
        return PPI_yoy                                                                        # 返回所有PPI环比和同比
    
    def PPI_by_PMI(self,verbose = 0):
        
        ######## 通过测试集上MSE最小来选择最优的建模开始时间 ########
        
        df = read_from_excel(self.workbook,"PMI数据")
        date2 = datetime.strptime(self.date,'%Y-%m-%d')                         # 预测时间格式标准化
        tp = pd.DatetimeIndex(df.date)
        df.index = tp
        df = df.drop('date',axis = 1)
        df = df.apply(pd.to_numeric,axis = 1)
        df_train = df[df.index < '2018-05-31']                                  # 设定2019年5月以前的数据作训练集
        df_test = df[df.index >= '2018-05-31']                                  # 设定2019年5月以后的数据作测试集   
        train_latest_start = '2015-05-01'                                       # 设定训练集的开始时间不能晚于2016年5月
        train_start_tp = tp[tp < train_latest_start]                            # 训练集开始时间集合         
        mse = 0                                                                 # 首先设定MSE=0
        for i in train_start_tp:                                                # 遍历训练集开始时间集合中的每一天
            train_set = df_train[df_train.index >= i]
            test_set = df_test
            model = smf.ols('y~PMI',train_set).fit()                          
            y_pred = model.predict(test_set.PMI)                                
            if mse == 0:
                mse = ((y_pred - test_set.y)**2).mean()
            elif (((y_pred - test_set.y)**2).mean() < mse ):
                mse = ((y_pred - test_set.y)**2).mean()
                start = i                                                       # 记录更优的建模开始时间
                linear_model_best = model                                       # 记录更优的回归模型
        print("从",start,"开始建模...")
        
        ######## 建立回归模型 ########
        
        df_final = df[(df.index >= start)&(df.index < date2)]
        reg_model = smf.ols('y~PMI',df_final).fit()                             # 建模    
        y_reg_pre = reg_model.params[0]+reg_model.params[1]*df.loc[self.date,'PMI']
        
        
        ######## 对回归残差拟合ARMA模型 ########
        
        df_res = pd.DataFrame(reg_model.resid)
        df_res.index = pd.DatetimeIndex(reg_model.resid.index,freq = 'MS')
        best_ARMA = choose_best_ARMA(df_res)
        res_pre = best_ARMA.forecast(steps = 1)[0]
        PPI_mom_pre = y_reg_pre+res_pre
        if verbose == 1:
            print('回归模型：')
            print(reg_model.summary())
            print('ARMA模型：')
            print(best_ARMA.summary())
        y = np.asarray(df[df.index<self.date]['y'])
        PPI_mom = pd.DataFrame(np.hstack((y,PPI_mom_pre)),columns=['PPI环比'],
                               index = range(y.shape[0]+1))                    # 最终PPI环比预测
        
        ######## 环比数据转为同比数据 ########  
        
        PPI_yoy = mom_to_yoy(PPI_mom)
        PPI_yoy.index = pd.date_range(start = df.index[0],periods = y.shape[0]+1,freq = 'M')
    
        return PPI_yoy
    
    
    

    

### 读取数据以及参数设定

In [8]:
wb = xlrd.open_workbook(os.path.join(os.getcwd(), 'PPI预测模型数据.xlsx'))# 打开Excel文件
industry_names = ['煤炭',
                  '石油和天然气开采业',
                  '石油加工炼焦',
                  '有色冶炼',
                  '化学原料及化学品',
                  '化学纤维',
                  '黑色金属矿采',
                  '黑色加工',
                  '农副',
                  '非金属矿物制品业']
date = '2020-07-01'

In [9]:
PPI_predict = PPI_model(wb,date)
# 用分行业法预测2020年6月PPI
PPI_pre1 = PPI_predict.PPI_by_Industry(industry_names)
PPI_pre1.tail(5)

即将输出各行业系数...
          系数            p值
0  -0.016240  4.605875e-02
1   0.023955  1.029618e-06
2   0.001452  2.880575e-01
3   0.045923  3.750700e-13
4   0.038232  1.785820e-06
5   0.116156  1.127356e-08
6   0.019049  3.494728e-02
7   0.012686  1.133891e-02
8   0.071743  1.001111e-20
9   0.054330  1.533108e-03
10  0.114700  7.240508e-12


,PPI环比,同比
2020-03-31,-1.000000,-1.496478
2020-04-30,-1.300000,-3.067820
2020-05-31,-0.400000,-3.648252
2020-06-30,0.400000,-2.971760
2020-07-31,0.324533,-2.461796


In [11]:
# 用PMI法预测2020年6月PPI
PPI_pre2 = PPI_predict.PPI_by_PMI()
PPI_pre2.tail()


从 2008-07-01 00:00:00 开始建模...


,PPI环比,同比
2020-03-31,-1.000000,-1.496478
2020-04-30,-1.300000,-3.067820
2020-05-31,-0.400000,-3.648252
2020-06-30,0.400000,-2.971760
2020-07-31,0.450676,-2.339156


### 模型效果可视化

In [12]:
df = read_from_excel(wb,'真实数据')
train = df[df.date < '2020-01-31'][['date','PPI同比']]
test = df[df.date >= '2020-01-31'][['date','PPI同比']]
pre_date = ['2020-01-01','2020-02-01','2020-03-01','2020-04-01','2020-05-01','2020-06-01']

pred = np.zeros(shape = (6,2))
for i,date in enumerate(pre_date):
    pred[i,0] = PPI_model(wb,date).PPI_by_Industry(industry_names).iloc[-1,-1]
    pred[i,1] = PPI_model(wb,date).PPI_by_PMI(industry_names).iloc[-1,-1]



即将输出各行业系数...
          系数            p值
0  -0.017888  4.056812e-02
1   0.026168  1.133425e-06
2   0.002815  2.379403e-01
3   0.042526  1.572904e-08
4   0.033692  1.714214e-04
5   0.121156  2.692429e-08
6   0.023739  1.575605e-02
7   0.011632  2.488423e-02
8   0.070169  2.395871e-18
9   0.061771  8.127844e-04
10  0.117438  5.648173e-11
从 2008-07-01 00:00:00 开始建模...
即将输出各行业系数...
          系数            p值
0  -0.017262  4.520770e-02
1   0.025977  1.094479e-06
2   0.002843  2.307371e-01
3   0.042989  8.218220e-09
4   0.034334  1.094924e-04
5   0.119352  2.405160e-08
6   0.023827  1.483754e-02
7   0.011806  2.197563e-02
8   0.070049  1.500901e-18
9   0.060832  8.637834e-04
10  0.117432  4.311509e-11
从 2008-07-01 00:00:00 开始建模...
即将输出各行业系数...
          系数            p值
0  -0.017570  3.764075e-02
1   0.025873  8.893776e-07
2   0.002881  2.198494e-01
3   0.043153  4.773123e-09
4   0.034627  7.169107e-05
5   0.118745  1.631697e-08
6   0.023708  1.438608e-02
7   0.011735  2.145560e-02
8   0.0701

In [13]:
pred_df = pd.DataFrame(pred,columns = ['分行业预测值','PMI预测值'])
pred_df['date'] = pre_date

In [14]:
import plotly
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)
trace1 = go.Scatter(
        x=train['date'],
        y=train['PPI同比'],
        name = '训练集真实值'
    )
trace2 = go.Scatter(
        x=test['date'],
        y=test['PPI同比'],
        name = '预测集真实值'
    )
trace3 = go.Scatter(
        x=pred_df['date'],
        y=pred_df['分行业预测值'],
        name = '分行业预测值', 
        mode="markers"
       )
trace4 = go.Scatter(
        x=pred_df['date'],
        y=pred_df['PMI预测值'],
        name = 'PMI预测值',  
        mode="markers"
       )

d = [trace1,trace2,trace3,trace4]

fig = go.Figure(data = d)
iplot(fig)